# Full Pipeline Notebook
This notebook consolidates all repository code (except the tests) into one workflow.

## Imports and utility functions

In [ ]:

import os
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
import requests

try:
    from entsoe import EntsoePandasClient
except ImportError:
    EntsoePandasClient = None

try:
    import holidays
except ImportError:
    holidays = None


In [ ]:

BASE_URL = "https://www.swissgrid.ch/dam/swissgrid/current/Data/excel/balancingenergy/"

def download_monthly_price(year: int, month: int, folder: Path = Path("data")) -> Path:
    """Download a monthly balancing energy price file. If download fails a dummy file is created."""
    folder.mkdir(exist_ok=True)
    filename = folder / f"{year}_{month:02d}_balancing_energy_prices.xlsx"
    url = f"{BASE_URL}{year}/{month:02d}/balancing_energy_prices_{year}_{month:02d}.xlsx"
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        filename.write_bytes(r.content)
        print(f"Downloaded {filename}")
    except Exception as e:
        print(f"Could not download {url}: {e}. Creating dummy data")
        dummy = pd.DataFrame({
            'Time': pd.date_range(f"{year}-{month:02d}-01", periods=96, freq='15min'),
            'PositivePrice': 0.0,
            'NegativePrice': 0.0,
        })
        dummy.to_excel(filename, index=False)
    return filename


In [ ]:

def load_price_file(path: Path) -> pd.DataFrame:
    df = pd.read_excel(path)
    df = df.rename(columns=str.strip)
    df['Time'] = pd.to_datetime(df['Time'])
    return df.set_index('Time')


def add_basic_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['hour'] = df.index.hour
    df['weekday'] = df.index.weekday
    df['month'] = df.index.month
    df['dayofyear'] = df.index.dayofyear
    df['lag_price'] = df['PositivePrice'].shift(1)
    df['rolling_mean_24h'] = df['PositivePrice'].rolling(window=24).mean()
    df['rolling_std_24h'] = df['PositivePrice'].rolling(window=24).std()
    return df


In [ ]:

def fetch_entsoe_load(start: datetime, end: datetime, token: str) -> pd.DataFrame:
    if EntsoePandasClient is None:
        raise RuntimeError('entsoe-py not installed')
    client = EntsoePandasClient(api_key=token)
    try:
        load = client.query_load(country_code='CH', start=start, end=end)
        return load.rename('load').to_frame()
    except Exception as e:
        print(f'Could not download ENTSO-E data: {e}. Returning dummy data')
        idx = pd.date_range(start, end, freq='H', inclusive='left')
        return pd.DataFrame({'load': 0.0}, index=idx)


def fetch_weather(start: datetime, end: datetime, lat: float, lon: float) -> pd.DataFrame:
    url = (
        'https://api.open-meteo.com/v1/forecast'
        f'?latitude={lat}&longitude={lon}&hourly=temperature_2m,wind_speed_10m'
        f'&start_date={start.date()}&end_date={end.date()}&timezone=UTC'
    )
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        data = r.json()
        times = pd.to_datetime(data['hourly']['time'])
        return pd.DataFrame({
            'temperature': data['hourly']['temperature_2m'],
            'wind_speed': data['hourly']['wind_speed_10m']
        }, index=times)
    except Exception as e:
        print(f'Could not download weather data: {e}. Returning dummy data')
        idx = pd.date_range(start, end, freq='H', inclusive='left')
        return pd.DataFrame({'temperature': 0.0, 'wind_speed': 0.0}, index=idx)


def fetch_holidays(start: datetime, end: datetime) -> pd.DataFrame:
    if holidays is None:
        raise RuntimeError('holidays library not installed')
    ch_holidays = holidays.CH(years=range(start.year, end.year + 1))
    idx = pd.date_range(start, end, freq='D')
    flags = [1 if day.date() in ch_holidays else 0 for day in idx]
    return pd.DataFrame({'holiday': flags}, index=idx)


In [ ]:

def merge_additional_data(prices: pd.DataFrame,
                          load: pd.DataFrame | None = None,
                          weather: pd.DataFrame | None = None,
                          holidays_df: pd.DataFrame | None = None) -> pd.DataFrame:
    df = prices.resample('H').mean()
    if load is not None:
        df = df.join(load, how='left')
    if weather is not None:
        df = df.join(weather, how='left')
    if holidays_df is not None:
        df = df.join(holidays_df, how='left')
    return df


## Download and prepare data

In [ ]:

end = datetime.utcnow()
start = end - timedelta(days=90)
price_frames = []
current = start.replace(day=1)
while current <= end:
    path = download_monthly_price(current.year, current.month)
    price_frames.append(load_price_file(path))
    if current.month == 12:
        current = current.replace(year=current.year + 1, month=1)
    else:
        current = current.replace(month=current.month + 1)
prices = pd.concat(price_frames).sort_index()


In [ ]:

entsoe_token = os.getenv('ENTSOE_TOKEN', '')
if entsoe_token:
    load = fetch_entsoe_load(prices.index.min(), prices.index.max() + pd.Timedelta(hours=1), entsoe_token)
else:
    print('ENTSOE_TOKEN not set. Using dummy load data')
    load = fetch_entsoe_load(prices.index.min(), prices.index.max() + pd.Timedelta(hours=1), entsoe_token)

weather = fetch_weather(prices.index.min(), prices.index.max() + pd.Timedelta(hours=1), lat=46.8, lon=8.3)
holidays_df = fetch_holidays(prices.index.min(), prices.index.max())


In [ ]:

merged = merge_additional_data(prices, load, weather, holidays_df)
features = add_basic_features(merged).join(merged[['load', 'temperature', 'wind_speed', 'holiday']])
features = features.dropna()


## Train model using all available data

In [ ]:

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

X = features[[
    'hour', 'weekday', 'month', 'dayofyear', 'lag_price',
    'rolling_mean_24h', 'rolling_std_24h', 'load',
    'temperature', 'wind_speed', 'holiday'
]]
y = features['PositivePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
model = GradientBoostingRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('MAE:', mean_absolute_error(y_test, y_pred))


This notebook demonstrates the full pipeline from data acquisition to model training in one place.